#### (1) 가산 잡음 처리

In [3]:
#노이즈 영상들을 합쳐서 평균 -> 노이즈 제거됨
#간단하지만 파워풀함 시험에 나올듯.? 몰루..
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
avg = np.zeros_like(image)
n = 0

for i in range(20):
    noise = np.random.randn(*image.shape).astype(np.float32) * 0.25 #표준편차 0.25
    noised = (image + noise).clip(0,1) #너무 큰거는 1로 작은거는 0으로
    avg += noised
    n += 1
    psnr1 =cv2.PSNR(image, noised,1)
    psnra =cv2.PSNR(image, avg/n,1)
    cv2.imshow('noise reduction: {:0.2f} -> {:0.2f} dB'.format(psnr1,psnra), np.hstack([noised,avg/n]))
    cv2.waitKey()
    cv2.destroyAllWindows()


#### (2) IIR Filter

In [5]:
# 기존에 있는 영상 90% + 새로들어온 영상(노이즈) 10% -> 노이즈가 줄어듬
# 밝기가 0에서 점점 밝아짐
# 메모리를 적게 쓰면서 가산잡음처럼 할 수 있음
# 완쪽, 가운데, 오른쪽 / 새로 들어온 영상, 지금까지 가산 평균 영상, IIR필터를 적용한 영상 

import cv2
import numpy as np

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
avg = np.zeros_like(image)
n = 0
filtered = np.zeros_like(image)

alpha = 0.9

for i in range(30):
    noise = np.random.randn(*image.shape).astype(np.float32) * 0.25
    noised =(image + noise).clip(0,1)
    filtered = filtered * alpha + noised * (1-alpha) 
    # filtered = cv2.normalize(filtered,None,0,1,cv2.NORM_MINMAX)
    avg += noised
    n += 1

    psnr1 =cv2.PSNR(image, noised,1)
    psnra =cv2.PSNR(image, avg/n,1)
    psnrf =cv2.PSNR(image, filtered,1)
    cv2.imshow('noise reduction: {:0.2f} -> {:0.2f} -> {:0.2f} dB'.format(psnr1,psnra,psnrf), \
        np.hstack([noised,avg/n,filtered]))
    cv2.waitKey()
    cv2.destroyAllWindows()

#### (3) Spatial Filter 1 - Moving Average

In [6]:
# 사진 1장으로 잡음을 줄일 수 있음
# 이웃화소가 천천히 변함 영상(낮은 주파수), 잡음(매우 높은 주파수) -> LPF하면 높은 주파수의 신호를 잃긴하지만 잡음을 잘잡음
image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)

blur = cv2.blur(noised,(11,11)) # 이웃하는 화소들의 평균을 취함
cv2.imshow('moving average', np.hstack([noised,blur]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (4) Spatial Filter 2 - Gaussian Blur

**Gaussian Kernel**

$G_i = \alpha * e^-{\frac{(i-(ksize-1)/2)^2}{2*\sigma^2}}, i=0,1,...,ksize-1$

$\alpha$: scale factor chosen s.t. $\sum_i G_i = 1$

**kernel = getGaussianKernel(ksize, sigma, ktype=CV_64F)**

-`ksize` : Aperture size. It should be odd ( `ksize mod 2 = 1` ) and positive.

-`sigma` : Gaussian standard deviation. If it is non-positive, it is computed from ksize as $\sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8$.

-`ktype` : Type of filter coefficients. It can be CV_32F or CV_64F .

In [7]:
# 가중치들이 정규분포의 모양을 가지고 있음 -> 가우시안 필터
# 가까운게 상관성이 있을 확률이 더 큼

gauss = cv2.GaussianBlur(noised, (11, 11), 0)

kernel = cv2.getGaussianKernel(11, 0)
filtered = cv2.filter2D(noised, -1, kernel)

cv2.imshow('filtered', np.hstack([blur,gauss,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (5) Spatial Filter 3 - Median Filter

In [8]:
# 페퍼(0)엔 솔트(255) 잡음에 강함
# 이웃하는 화소들의 밝기를 줄을 세워서 중간값을 취함 (잡음인 0 or 255이 한 두개면 잘처리함)

median_blur = cv2.medianBlur(noised, 5)
cv2.imshow('filtered', np.hstack([blur,median_blur]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (6) Spatial Filter 4 - Bilateral Filter

In [12]:
# 잡음에 의해 외곡이 조금 되었지만 원본 데이터를 가짐
# 색깔이 비슷하면 멀리있더라도 웨이트를 더 줌
# 가우시안 필터 + 색깔이 비슷하면 웨이트 더 줌 (가까워도 색깔이 많이 다르면 가중치가 작음)

bilat = cv2.bilateralFilter(noised,11,0,0)
cv2.imshow('filtered', np.hstack([blur,bilat]))
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
#왼쪽 위 MA 시계방향 순으로
cv2.imshow('MA-Gauss-Bilat-Median (clockwize)', np.vstack([np.hstack([blur,gauss]),np.hstack([median_blur,bilat])]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (7) Adaptive Wiener Filter

In [11]:
# 영상의 스펙트럼을 측정하고 화소하나마다 인접하는 화소들의 주파수를 확인하고 cutup(임계값)을 조절
# 시간이 오래 걸림 
import cv2
import numpy as np

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)
filtered = noised.copy()

y, x = image.shape[:2]

for i in range(11,y-11):
    for j in range(11,x-11):
        for k in range(3):
            block = noised[i-11:i+12,j-11:j+12,k]
            mf = np.mean(block)
            sig = np.std(block)
            filtered[i,j,k] = mf + sig**2/(sig**2+0.01)*(noised[i,j,k]-mf)

cv2.imshow('filtered', np.hstack([noised,filtered]))
cv2.waitKey()
cv2.destroyAllWindows()

#### (8) Measure of matching

In [11]:
# 신호처리를 한 영상이 얼마나 잘됐는지 수치(오리지널 영상과 얼마나 비슷한지)
# 화소 대 화소의 차이의 제곱을 로그 -> 이것은 정확하지 않을 수 있음 
# 예를 들어 같은 영상을 1화소만 옆으로 옮기면 에러가 크지만 눈으로 보면 차이가 별로 없음
# 다른 객관적 지표들이 있음
# Cross Correlation : 픽셀끼리 곱하면 양수는 양수끼리 음수는 음수끼리 곱하기 때문에 값이 커짐
# 같은 영상 화소값이 0 ~ 255 , 0 ~ 125 값이면 Cross Correlation은 값이 큼 하지만 그냥 픽셀와이즈하는 건 안좋음
# but 0을 중심으로 50을 중심으로 화소값이 움직이면 CCORR은 비효율적 -> CCOEFF로 대체 

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCORR','cv2.TM_SQDIFF', 
            'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']

import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread('../data/Lena.png').astype(np.float32) / 255
noise = np.random.randn(*image.shape).astype(np.float32) * 0.1
noised = (image + noise).clip(0,1)

median_blur = cv2.medianBlur(noised, 5)
bilat = cv2.bilateralFilter(noised,11,0,0)

psnr = cv2.PSNR(image, median_blur, 1)
print(psnr,'dB')

for k in range(6):
    res = cv2.matchTemplate(image, median_blur, eval(methods[k])) #
    if k <= 3:
        res = res/(image.shape[0]*image.shape[1]*image.shape[2])
    if k == 2:
        psnr1 = -10*np.log10(res[0][0])
        print(psnr1,'dB', end=' ')
    print(methods[k], res)


27.792012816405276 dB
cv2.TM_CCOEFF [[0.0310449]]
cv2.TM_CCORR [[0.3047682]]
27.791971791655655 dB cv2.TM_SQDIFF [[0.00166266]]
cv2.TM_CCOEFF_NORMED [[1.23873368e-06]]
cv2.TM_CCORR_NORMED [[0.99728286]]
cv2.TM_SQDIFF_NORMED [[0.00544066]]
